Here is an exercise to understand the Logistic Regression algorithm used in machine learning for supervised classification tasks. 

An overview of the mathematics used can be found on Andrew Ng's Coursera course for deep learning. This exercise borrows heavily, although not exclusively, from that. The dataset we will be working with today is the titanic survival dataset found on Kaggle where passengers are labeled 1 if they survived and 0 if they did not and we are asked to predict which passangers lived given a dataset without survival labels. I chose this data because everyone has heard of the data set and I had easy access to it. The data is really not important for this exercise, as long as it is generic enough to generalize to similar use cases. 

### Important:
The goal here is to understand the algorithm rather than showcase the Data Science and exploration processes that occur both before and after the point at which we decide on and build an algorithm. There are many steps left out (on purpuse) like data exploration, model and cross validation, feature selection and much more. 

Lets first take a look at a standard implimentation from Sklearn that is prepackaged, then build out the algorithm the long way (notably inefficient, then we will use Linear Algebra to improve efficiency for scalability. 





# Logistic Regression using Sklearn (Prepackaged)

In [4]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [5]:
# import data
df1 = pd.read_csv('titanic_train.csv')
df2 = pd.read_csv('titanic_test.csv')

# do same preprocessing on fare
df1.Fare.fillna(14.45, inplace=True)
df2.Fare.fillna(14.45, inplace=True)
df1['fare'] = df1['Fare']/513
df2['fare'] = df2['Fare']/513

# first class indicator
df1['first_class'] = (df1['Pclass'] == 1).astype(int)
df2['first_class'] = (df2['Pclass'] == 1).astype(int)

# female indicator
df1['female_flag'] = (df1['Sex'] == 'female').astype(int)
df2['female_flag'] = (df2['Sex'] == 'female').astype(int)

In [6]:
X_train = df1.loc[:, ['fare', 'first_class', 'female_flag']]
y_train = df1['Survived']

model = LogisticRegression(random_state=333, solver='liblinear')
model.fit(X_train, y_train)



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=333, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

# Logistic Regression from Scratch (First Way)

In [1]:
import csv
import math

def import_data(pathname, train_or_test):
    with open(pathname, newline='') as csvfile:
        filereader = csv.reader(csvfile, delimiter=',')
    
        survived = list()
        fare = list()
        first_class = list()
        female_flag = list()

        for idx, row in enumerate(filereader):
            if idx == 0:
                fare_ix = row.index('Fare')
                fc_ix = row.index('Pclass')
                sex_ix = row.index('Sex')
                if train_or_test == 'train':
                    surv_ix = row.index('Survived')
            if idx != 0:
                if train_or_test == 'train':
                    survived.append(int(row[surv_ix]))
                else:
                    survived.append('?')
                first_class.append(int(row[fc_ix]=='1'))
                try:
                    fare.append(float(row[fare_ix]))
                except:
                    fare.append(14.45) #training data median fare
                female_flag.append(int(row[sex_ix]=='female'))

                
    return survived, fare, first_class, female_flag

def sigmoid_function(x):
    return 1/(1 + math.exp(-x))


In [2]:
# import data
survived_tr, fare_tr, first_class_tr, female_flag_tr = import_data('titanic_train.csv', 'train')
survived_tst, fare_tst, first_class_tst, female_flag_tst = import_data('titanic_test.csv', 'test')

In [3]:
# train logistic regression model

w1, w2, w3, b = 0, 0, 0, 0 #initialize
learning_rate = .06

for k in range(10_001):
    J, dw1, dw2, dw3, db = 0, 0, 0, 0, 0 #initialize 
    for i in range(len(survived_tr)):
        y  = survived_tr[i]
        z  = (w1*(fare_tr[i]/513) + #min max scale fare using train max
              w2*first_class_tr[i] + 
              w3*female_flag_tr[i] + 
              b)
        a  = sigmoid_function(z)
        dz = a - y
        J   += -(y*math.log(a) + (1-y)*math.log(1-a))
        dw1 += (fare_tr[i]/513)*dz 
        dw2 += first_class_tr[i]*dz
        dw3 += female_flag_tr[i]*dz
        db  += dz

    dw1 /= len(survived_tr)
    dw2 /= len(survived_tr)
    dw3 /= len(survived_tr)
    db  /= len(survived_tr)
    J   /= len(survived_tr)

    w1 -= learning_rate*(dw1)
    w2 -= learning_rate*(dw2)
    w3 -= learning_rate*(dw3)
    b  -= learning_rate*(db)
    
    if k%1_000 == 0:
        print(
            'iteration: ',k,
            ' loss: ',round(J,3),
            '|| w1={}, w2={}, w3={}, b={}'
                .format(round(w1,3),round(w2,3),round(w3,3),round(b,3))
             )

iteration:  0  loss:  0.693 || w1=-0.001, w2=-0.004, w3=-0.005, b=-0.015
iteration:  1000  loss:  1.776 || w1=-0.193, w2=-0.675, w3=-0.945, b=-3.651
iteration:  2000  loss:  2.083 || w1=-0.224, w2=-0.762, w3=-1.059, b=-4.358
iteration:  3000  loss:  2.26 || w1=-0.241, w2=-0.809, w3=-1.12, b=-4.765
iteration:  4000  loss:  2.384 || w1=-0.253, w2=-0.841, w3=-1.16, b=-5.052
iteration:  5000  loss:  2.48 || w1=-0.262, w2=-0.865, w3=-1.19, b=-5.273
iteration:  6000  loss:  2.558 || w1=-0.269, w2=-0.883, w3=-1.215, b=-5.454
iteration:  7000  loss:  2.624 || w1=-0.275, w2=-0.899, w3=-1.234, b=-5.606
iteration:  8000  loss:  2.681 || w1=-0.28, w2=-0.913, w3=-1.251, b=-5.738
iteration:  9000  loss:  2.731 || w1=-0.285, w2=-0.924, w3=-1.266, b=-5.854
iteration:  10000  loss:  2.776 || w1=-0.289, w2=-0.935, w3=-1.279, b=-5.958


# Logistic regression vectorized (second way)

In [1]:
#required imports
# import pandas as pd
# import numpy as np

In [7]:
def initialize_weights(x_dim):
    weights = np.zeros((x_dim, 1))
    b = 0.0
    return weights, b

In [8]:
def sigmoid_np(z):
    return 1/(1+np.exp(-z))

In [12]:
def forward_pass(w, b, X, Y):
    m = X.shape[0]
    
    Y = np.array(Y).reshape(1,Y.shape[0])
    A = sigmoid_np(np.dot(w.T, X.T) + b)
    cost = (-1/m)*np.sum(Y*np.log(A) + (1-Y)*np.log(1-A))
    
    dw = (1/m)*(np.dot(X.T, (A-Y).T))
    db = (1/m)*np.sum(A-Y)
    
    return cost, dw, db

In [21]:
def gradient_descent(w, b, X, Y, steps, alpha):
    cost_curve = list()
    
    for i in range(steps+1):
        
        cost, dw, db = forward_pass(w, b, X, Y)
        
        w -= alpha*dw
        b -= alpha*db
        
        if i % 2_000 == 0:
            cost_curve.append(cost)
            print('Cost at iteration {}: {}'.format(i, cost))
            
    return w, b, cost_curve

In [24]:
def train_model(X_train, Y_train, steps=10_000, alpha=.06):
    
    #initialize
    weights, b = initialize_weights(X_train.shape[1])
    
    #optimize
    w, b, cost_curve = gradient_descent(
        weights, b, X_train, Y_train, steps=steps, alpha=alpha)
    
    return w, b, cost_curve
    

In [25]:
vect_w, vect_b, cost_curve = train_model(X_train, y_train)

Cost at iteration 0: 0.6931471805599454
Cost at iteration 2000: 0.4773103297648902
Cost at iteration 4000: 0.47687641624497756
Cost at iteration 6000: 0.47680513332820845
Cost at iteration 8000: 0.47675104507479743
Cost at iteration 10000: 0.47670837546033246


In [31]:
# def predict(w, b, X):
#     m = X.shape[0]
    
#     Y_preds = np.zeros((1, m))
#     w = w.reshape((X.shape[1], 1))
    
#     pred = sigmoid_np(np.dot(w.T, X.T) + b)
    
#     for i in range(pred.shape[1]):
#         Y_preds[0,i] = int(pred[0][i] > 0.5)
        
#     return Y_preds

# Results:

In [33]:
X_train.head(3)

,fare,first_class,female_flag
0,0.014133,0,0
1,0.138954,1,1
2,0.015448,0,1


Let's take a look at how closely the different methodologies converged to a solution. w1 is the weight associated with fare or ticket price. w2 is the weight for the indicator variable if a passanger was in a first class cabin. w3 is the weight for the indicator if a passanger on the titanic was a female. b is the intercept (bias) term. 

### Logistic regression using sklearn

In [287]:
print('w1: ', model.coef_[0][0])
print('w2: ', model.coef_[0][1])
print('w3: ', model.coef_[0][2])
print('b: ', model.intercept_[0])

w1:  0.6759643072666862
w2:  1.3928607842089231
w3:  2.4987085910576283
b:  -1.8429052148114269


### Logistic regression from scratch using loops (first way)

In [288]:
print('w1: ', w1)
print('w2: ', w2)
print('w3: ', w3)
print('b: ', b)

w1:  0.6778369918314936
w2:  1.4953049780875323
w3:  2.6177557917653225
b:  -1.9325952928155834


### Logistic regression from scratch vectorized (second way)

In [32]:
print('w1: ', vect_w[0][0])
print('w1: ', vect_w[1][0])
print('w1: ', vect_w[2][0])
print('b: ', vect_b)

w1:  0.6778369918314934
w1:  1.4953049780875325
w1:  2.6177557917653225
b:  -1.9325952928155836


The implimentations from scratch match exactly and show two different ways of arriving at the same answer. The weights the algorithm optimized on are very similar to the Sklearn implimentation of Logistic Regression but slightly different because they use different stopping criteria and different optimization parameters. Extremely close for all intents and purposes. 

In practice we often just impliment the the standard Instantiate, fit, predict pipeline for machine learning without too much thought about what goes inside that black box. Running through exercises like this not only allows you to understand how you can modify algorithms as we get more advanced but also will allow us to explain how we are arriving at solutions to all audiences from completely novice to extremely technical. 

In [ ]:
!git add gradient_descent_from_scratch.ipynb

In [ ]:
!git commit -m "finished vectorization and added more comment and results"

In [ ]:
!git push -u origin master